# Challenge FTP

Ce challenge a pour but d'annalysée et comprendre les trames FTP

Pour commencer on importe bien évidemment la librairie "Scapy".

In [7]:
from scapy.all import *
ftp = rdpcap("ftp-total.pcapng")

## 1. Filtrage du Fichier

Pour récupérer les trames FTP on peut uttilisée les ports. FTP est uttilisée uniquement depuis les ports **20** ou **21**
On vérifie donc uniquement si le port **source** ou **destination** est le **20** ou **21**.

On tape donc en filtre sur **Wireshark**

`tcp.port == 21 or tcp.port == 20 || ftp`

Cela nous affichie bien toute les trames FTP avec les SYN, SYN/ACK et ACK

![](https://i.ibb.co/vvsNzhn4/Capture-d-cran-2025-06-01-005641.png)

## 2. Analyse visuelle de la capture

Lorsque l'on regarde la capture FTP, on obtient plein d'information comme l'ip du serveur (**10.2.12.16**) ou aussi l'ip du client (**10.2.4.3**). On voit également plusieur noms de paquets comme USER, PASS, SYST, LIST et pleins d'autre.

Voici la liste de certaine de ces requêtes applicative :

* **USER** : Donne le nom d'uttilisateurs
* **PASS** : Donne le mot-de-passe
* **SYST** : Demande le système d'exploitation du serveur
* **PORT** : Les 4 premier chiffre correspondent à l'adresse ip (ici 10,2,4,3) et les deux dernier corresponde a deux chiffre P1 et P2 permetant de faire un calcul qui donnera le port (*P1 * 256 + P2*)
* **QUIT** : Permet de quitter la connexion

## 3. Extraction du fichier transféré

Dans la connexion, un fichier y est transferée. On peut le récupérée de la manière suivante :

* Allez dans `Fichier > Exporter Objet > FTP-DATA`
* Cela devrez vous ouvrir un fenêtre contenant un fichier avec comme nom **ftpdoc.odt**
* Faite `Enregistrer`

Et voilà le fichier transfèrée vous appartient !

## 4. Développement d’un “Exploit” en Python

Voici un code python qui regarde si il y a une connexion FTP qui se réalise

Ce premier code permet de récupéré nom d'uttilisateur, mot-de-passe et l'addresse ip et le port du client depuis un fichier `.pcapng`

In [13]:
def parse_ftp_from_pcapng(packets):

    for pkt in packets:
        # On filtre les paquets TCP contenant des données FTP
        if pkt.haslayer(TCP) and (pkt[TCP].dport == 21 or pkt[TCP].sport == 21):
            if Raw in pkt:
                try:
                    payload = pkt[Raw].load.decode('utf-8', errors='ignore')

                    # Recherche des commandes FTP
                    if "USER" in payload or "PASS" in payload:
                        
                        if "USER" in payload:
                            user = re.search(r"USER (.+?)\r\n", payload)
                            if user:
                                print(f"Nom d'utilisateur : {user.group(1)}")

                        if "PASS" in payload:
                            passwd = re.search(r"PASS (.+?)\r\n", payload)
                            if passwd:
                                print(f"Mot de passe : {passwd.group(1)}")

                except Exception as e:
                    print("Erreur lors du traitement d’un paquet:", e)

parse_ftp_from_pcapng(ftp)

Nom d'utilisateur : touriste
Mot de passe : 3aboqphie=3qbc!


Et voici le même code que ci-dessus mais qui regarde directement dans le réseaux

In [ ]:
def parse_ftp_packet(pkt):
    if pkt.haslayer(TCP) and (pkt[TCP].dport == 21 or pkt[TCP].sport == 21):
        if pkt.haslayer(Raw):
            try:
                payload = pkt[Raw].load.decode('utf-8', errors='ignore')

                if "USER" in payload or "PASS" in payload or "PORT" in payload:

                    if "USER" in payload:
                        user = re.search(r"USER (.+?)\r\n", payload)
                        if user:
                            print(f"Nom d'utilisateur : {user.group(1)}")

                    if "PASS" in payload:
                        passwd = re.search(r"PASS (.+?)\r\n", payload)
                        if passwd:
                            print(f"Mot de passe : {passwd.group(1)}")

            except Exception as e:
                print("Erreur lors du traitement d’un paquet:", e)

# Lancer le sniff réseau
print("Démarrage de l'écoute du trafic FTP sur le réseau...")
sniff(filter="tcp port 21", prn=parse_ftp_packet, store=0)
